# Antigen Random Surface Residue Detection
(Using PyMol)

In [1]:
from pymol import cmd
from random import sample
import pandas as pd

In [2]:
def find_random_surface_residues(file, percentage = 0.25):
    cmd.load(file)
    ## Finds atoms on the surface of a protein
    ## Logic from: https://pymolwiki.org/index.php/FindSurfaceResidues
    cutoff = 2.0
    tmpObj = cmd.get_unused_name("_tmp")
    cmd.create(tmpObj, "(all) and polymer", zoom=0)
    cmd.set("dot_solvent", 1, tmpObj)
    cmd.get_area(selection=tmpObj, load_b=1)
    cmd.remove(tmpObj + " and b < " + str(cutoff))
    cmd.select("exposed_atoms", "(all) in " + tmpObj)
    cmd.delete(tmpObj)
    ## Get a list of residue numbers and then subset it
    surface_residues = set()
    cmd.iterate('exposed_atoms', "surface_residues.add(resi)", space=locals())
    # surface_residues = list(surface_residues)
    k = int(len(surface_residues) * percentage)
    surface_residues_sample = list(sample(surface_residues, k))
    surface_residues_sample = [int(x) for x in surface_residues_sample]
    surface_residues_sample.sort()
    ## Reintialize Everything
    cmd.reinitialize(what='everything')
    return surface_residues_sample

In [3]:
## Load in antigens
antigens = pd.read_excel('../../Experiments.xlsx', sheet_name='Antigens')

In [4]:
for index, antigen in antigens.iterrows():
    # print(antigen['antigen_id'])
    antigen_pdb_path = antigen['antigen_pdb_path_renumbered']
    relative_antigen_pdb_path = f"../../{antigen_pdb_path}"
    surface_resn = find_random_surface_residues(relative_antigen_pdb_path, percentage = 0.25)
    
    print(f"{antigen['antigen_id']}: `{','.join(map(str, surface_resn))}")

C:\Users\Colby\AppData\Local\Temp\ipykernel_32460\925230197.py:18: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  surface_residues_sample = list(sample(surface_residues, k))


YP_308669.1: `112,115,117,121,122,126,135,136,139,140,146,148,159,160,174,175,178,182,195,199,206,209,213,217,219,222,229,239,244,249,265,266,267,268
WYN03019.1: `114,118,120,121,133,134,139,146,149,153,155,159,160,162,163,164,171,186,187,193,197,201,204,206,212,213,216,227,237,242,244,249,264,266
AAT73273.1: `115,118,119,120,121,122,123,128,132,141,143,146,149,153,169,174,176,182,186,192,193,197,199,203,216,217,222,228,231,232,234,236,241,248


## Visualize Random Surface Residues in py3Dmol

In [18]:
import py3Dmol

In [19]:
## Open PDB File
with open(pdb_path) as ifile:
    system = "".join([x for x in ifile])

## Create View
view = py3Dmol.view(width=800, height=500)
view.addModel(system, 'pdb')

## Define chains to show and their colors
chains_residues_to_highlight = {
    'A': surface_resn
}

chain_colors = {
    'A': "green"
}

## List of chains to hide (e.g., chain 'B')
# chains_to_hide = ['C', 'D', 'Y', 'Z']
chains_to_hide = []

# view.setStyle({"cartoon": {'color': 'black'}})
    
## Hide the specified chains
for chain_id in chains_to_hide:
    view.setStyle({'cartoon': 'off',
                   'stick': 'off',
                   'sphere': 'off',
                   'line': 'off',
                   'cross': 'off'},
                  {'model': -1,
                   'chain': chain_id})

## Set the style on the non-hidden data
for chain_id, residues_to_highlight in chains_residues_to_highlight.items():
    ## Color the chains by chain ID
    view.setStyle({'model': -1,
                   'chain': chain_id},
                  {'cartoon': {'color': chain_colors[chain_id]}})
    
    for residue_id in residues_to_highlight:
        ## Highlight specified residues in red for each chain
        view.addStyle({'model': -1,
                       'chain': chain_id,
                       'resi': residue_id},
                      {'stick': {'color': 'red',
                                 'radius': 1}})

## Zoom to fit the structure and show
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol